# Libraries

In [1]:
import  pandas as pd
import  numpy as np
import  plotly.express as px
import  matplotlib.pyplot as plt
import  plotly.graph_objs as go

from    sklearn.model_selection import train_test_split

In [2]:
RANDOM_SEED = np.random.seed(0)

In [3]:
cd ..\

c:\Users\victo\Downloads\00. Estudo Data Science\aps_failure_truck_prediction


c:\Users\victo\Downloads\00. Estudo Data Science\aps_failure_truck_prediction\venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Train

In [4]:
df_air_sys_prev = pd.read_csv('data\\original_data\\air_system_previous_years.csv',
                              encoding='utf-8',
                            #   na_values=['na']
                              )
df_air_sys_prev.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,neg,76698,na,2130706438,280,0,0,0,0,0,...,1240520,493384,721044,469792,339156,157956,73224,0,0,0
1,neg,33058,na,0,na,0,0,0,0,0,...,421400,178064,293306,245416,133654,81140,97576,1500,0,0
2,neg,41040,na,228,100,0,0,0,0,0,...,277378,159812,423992,409564,320746,158022,95128,514,0,0
3,neg,12,0,70,66,0,10,0,0,0,...,240,46,58,44,10,0,0,0,4,32
4,neg,60874,na,1368,458,0,0,0,0,0,...,622012,229790,405298,347188,286954,311560,433954,1218,0,0


In [5]:
df_train, df_validation = train_test_split(df_air_sys_prev, train_size=0.8, random_state=RANDOM_SEED)

In [6]:
df_train.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
5778,neg,60660,na,266,230,0,0,0,0,0,...,273744,145088,311782,371428,371870,381522,700624,9062,0,0
5287,neg,3412,na,86,76,0,0,0,0,0,...,9260,4462,11122,9132,11944,15364,70760,1712,0,0
57167,neg,29664,na,0,na,0,0,0,0,0,...,409912,222004,393608,182968,97552,45586,30968,268,0,0
29516,neg,50164,na,362,320,0,0,0,0,0,...,371442,196746,467836,388676,315952,231396,273286,1880,0,0
58610,neg,90754,na,0,na,0,0,0,0,0,...,910918,494190,1094292,933936,485084,146864,70744,12,0,0


In [7]:
df_validation.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
3048,neg,61620,na,2130706432,1208,0,0,0,0,0,...,427716,218536,442570,425574,461636,293122,324408,12410,0,0
19563,neg,551264,na,na,na,na,na,0,0,0,...,8838538,2117792,4444266,2836634,127750,10950,636,0,na,na
58303,neg,946,2,50,50,0,0,0,0,0,...,5888,2340,4684,4826,4642,16994,286,0,0,0
8870,neg,256688,na,na,na,na,na,0,0,1262,...,5605900,1759242,2163574,999712,646000,1140880,21712,0,na,na
40228,neg,61558,na,704,648,0,0,0,0,0,...,964318,458424,708748,397190,192778,93988,44716,6,0,0


# Test

In [8]:
df_air_sys_present = pd.read_csv('data\\original_data\\air_system_present_year.csv',
                                 encoding='utf-8',
                                #  na_values=['na']
                                 )
df_air_sys_present.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,neg,60,0,20,12,0,0,0,0,0,...,1098,138,412,654,78,88,0,0,0,0
1,neg,82,0,68,40,0,0,0,0,0,...,1068,276,1620,116,86,462,0,0,0,0
2,neg,66002,2,212,112,0,0,0,0,0,...,495076,380368,440134,269556,1315022,153680,516,0,0,0
3,neg,59816,na,1010,936,0,0,0,0,0,...,540820,243270,483302,485332,431376,210074,281662,3232,0,0
4,neg,1814,na,156,140,0,0,0,0,0,...,7646,4144,18466,49782,3176,482,76,0,0,0


In [9]:
df_test = df_air_sys_present.copy()

In [10]:
df_train.to_csv('data\data_split\df_train.csv',           encoding='utf-8', sep=',', index=False)
df_validation.to_csv('data\data_split\df_validation.csv', encoding='utf-8', sep=',', index=False)
df_test.to_csv('data\data_split\df_test.csv',             encoding='utf-8', sep=',', index=False)